<a href="https://colab.research.google.com/github/ETGrif/ecolabel-translator/blob/JessPress0-patch-1/LLM_Front_End_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project Structure

In [ ]:
/eco_label_translator
    ├── app.py               # Main Flask application
    ├── templates/
    │   ├── home.html        # Homepage template
    │   ├── results.html     # Results page template
    │   └── chat.html        # Chat interface template
    ├── static/
    │   └── style.css        # CSS for styling

app.py

Main Flask application that will handle routes for the front end

* Home Route ( /)
  - Displays home where users can search for label/start a chat
  - home.html displays the search form and chat start button
* Search Results ( /results)
  - Processes eco-label search and displays relevant info
  - Gets label from query param (label)
  - Looks up label info using get_label_data(label) function (OR WHATEVER THE B.E. IS USING!!!... just a placeholder for now)
  - Passes lable and retrieve data to results.html for display
  - Displays data, with option for more info
* Details ( /get_resp)
  - Generates/displays more detailed explanation of eco-label
  - Receieves label name via POST request
  - Requests a detailed explanation from GPTManager
  - Displayes details on results.html
* Initialize Chat ( /init_chat)
  - Creates new chat session/redirects to chat page
  - Calls chat_manager.create_chat() to start new session and generate chat_id
  - Redirects to chat page for this session with chat_id (redirect route: /chat/chat_id> )
* Chat Interface ( /chat/chat_id>)
  - Manages chat interface where users can send/receive messages
  - Retrieves chat messages for the unique chat_id
  - If POST request received, processes user message/gets response from ChatManager
  - Renders caht page with message history
* End Chat ( /terminate)
  - Does what it says
  - Receives chat_id and ends session in ChatManager
  - Redirects to homepage (redirect route: / )




In [ ]:
from flask import Flask, render_template, request, redirect, url_for
import requests  # For making HTTP requests to backend API
from some_module import ChatManager, DBManager, GPTManager  # Assuming these are
# implemented in some_module

app = Flask(__name__)

# Initializing managers
db_manager = DBManager()
chat_manager = ChatManager()
gpt_manager = GPTManager()

@app.route('/') # Defines route for homepage
def home():
    """Renders the homepage with a search form and a button to initiate chat."""
    return render_template('home.html')

@app.route('/results', methods=['GET']) # Defines a route at results, which
# accepts GET requests
def results():
    """Handles the results page that shows the eco-label information."""
    label = request.args.get('label')  # Gets the 'label' query parameter from the URL
    if not label:
        return redirect(url_for('home'))  # If label missing, redirects to home

    # Send a GET request to the backend API to get eco-label information
    try:
        response = requests.get(f'http://localhost:5000/search?label={label}') #
        response.raise_for_status()  # Raises an error for unsuccessful status
        # codes (404/500)
        data = response.json()  # Converts JSON response from backend to python dict
    except requests.exceptions.RequestException as e: # Catches errors in HTTP
    # request, logs error, then sets data to None if there is a problem
        print(f"Error fetching data: {e}")
        data = None  # If an error occurs, set data to None

    return render_template('results.html', label=label, data=data) #Renders results
    # template, passes in 'label' and 'data' (or None if there was an error)

@app.route('/init_chat', methods=['POST']) # Defines route that accepts POST requests
def init_chat():
    """Initializes a new chat session and redirects to the chat page."""
    chat_id = chat_manager.create_chat()  # Initializes a chat session in ChatManager
    # returns a unique chat ID
    return redirect(url_for('chat', chat_id=chat_id)) # Redirects user ot chat route

@app.route('/chat/<chat_id>', methods=['GET', 'POST']) # Defines a dynamic route
# where chat_id is passed as part of the URL. Accepts both GET and POST requests.
def chat(chat_id):
    """Handles chat interface for ongoing chat sessions."""
    if request.method == 'POST':
        message = request.form.get('message')  # User's message from the form
        response = chat_manager.get_resp(chat_id, message)  # Get response from ChatManager
        # (for given 'chat_id' and 'message')
        return render_template('chat.html', chat_id=chat_id, messages=response['messages'])
        # For get requests the above line just renders chat.html with chat ID
    return render_template('chat.html', chat_id=chat_id)

@app.route('/terminate', methods=['POST']) # Defines route to terminate, accepts POST
def terminate_chat():
    """Terminates an ongoing chat session and redirects to the homepage."""
    chat_id = request.form.get('chat_id')
    chat_manager.terminate_chat(chat_id)  # Terminates chat in ChatManager
    return redirect(url_for('home')) # Redirects back home

if __name__ == '__main__':
    app.run(debug=True)


home.html

HTML template for the homepage where users enter the eco-label name

* Eco-label search
  - User types label into search box on homepage and clicks search
  - Form submits to /results using a GET request wit hlabel name sent as query param (label)
  - B.E. uses label to look up info, returns results on results.html
* Start new chat session
  - User clicks "Start a Chat" button
  - Form submits to /init_chat route with POST request
  - B.E. initializes a new chat, redirects user to chat.html for that session, using unique chat_id


In [ ]:
<!DOCTYPE html> # Declares HTML doc
<html lang="en"> # Sets language to English
<head> # Metadata
    <meta charset="UTF-8"> # Character encoding setting
    <meta name="viewport" content="width=device-width, initial-scale=1.0"> # Page
    # scaling for different devices
    <title>Eco-Label Translator</title> # Sets title that is displayed in browser tab
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}"> # Links to CSS for style
</head>
<body> # Main display contents
    <div class="container"> # Container to center the content and apply styles
        <h1>Eco-Label Translator</h1> # Heading that shows title of app
        <form action="{{ url_for('results') }}" method="get"> # Defines form that will
        # submit to /results using GET request. The url_for part generates URL for
        # results route
            <input type="text" name="label" placeholder="Enter eco-label name" required> # Textbox
            # for users to enter the name of the label. name = 'label' makes input available
            # as query param, which Flask uses in request.args.get('label')
            <button type="submit">Search</button> # Button that submits form (sends
            # label to /results)
        </form>

        <!-- Form to start a new chat session -->
        <form action="{{ url_for('init_chat') }}" method="post"> # Form that submits to
        # /init_chat via POST req to create new chat sesh
            <button type="submit">Start a Chat</button> # Button to submit form (if clicked,
            # triggers init_cat to create new sesh and redirect to chat page)
        </form>
    </div>
</body>
</html>

results.html

HTML template for the results page (displays the fetched eco-label data)

* Displaying Info
  - If info available, displays description/EPA recommendation
  - If no data found, shows message that says so
* Requesting Details
  - Button for more info ("Get Detailed Explanation")
  - Form submits to /get_resp, includes label as hidden input
* Navigating Back
  - "Go back" button to return home

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Results for {{ label }}</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <div class="container">
        <h1>Results for "{{ label }}"</h1>

        {% if data %} # If data exists, display results
            <div class="results">
                <p><strong>Description:</strong> {{ data['description'] }}</p> # Displays
                # description of eco-lable (retrieved from B.E.)
                <p><strong>EPA Recommendation:</strong> {{ data['epaRecommendation'] }}</p> # Shows
                # EPA recommendations/other attributes for the label

                <!-- Form to get a detailed explanation from GPTManager -->
                <form action="{{ url_for('get_resp') }}" method="post"> # Form that posts
                # to /get_resp to request detailed explaination
                    <input type="hidden" name="label" value="{{ label }}"> # Passes
                    # label as hidden field, so backend knows which label to get
                    # explanation for (basically jsut preserves the searched label
                    # without making the user re-enter it)
                    <button type="submit">Get Detailed Explanation</button> # Button
                    # to submit form/get more details
                </form>
            </div>
        {% else %} # If data doesn't exist, displays message saying not found
            <p>No data found for this label.</p>
        {% endif %}

        <a href="{{ url_for('home') }}" class="back-link">Go back</a>
    </div>
</body>
</html>

chat.html

Chat Interface: For users to engage in a conversation with the backend

* Message display  
  - List of messages passed from Flask to chat.html as messages
  - rendered dynamically using Jinja2 (HTML page not static. It's generated with live data passed from B.E. using Jinja2)
* Send messages to
  - When form is submitted, B.E. gets the message, uses ChatManger to generate response, reloads chat.html with updated response
* Chat termination
  - As it sounds


In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Chat Assistant</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <div class="container"> # Container for chat interface
        <h2>Eco-Label Chat Assistant</h2> # Displayed heading
        # Messages Display:
        <div id="messages"> # Loops over messages given by B.E.
            {% for message in messages %} # Each message displays sender's name and the text
                <p><strong>{{ message['sender'] }}:</strong> {{ message['text'] }}</p>
            {% endfor %}
        </div>
        # Message Form:
        <form action="{{ url_for('chat', chat_id=chat_id) }}" method="post"> # Defines form
        # that submits to chat route for that session/chat_id
            <input type="text" name="message" placeholder="Type your message..." required> # For
            # user input (obviously) (also sets field to mandatory)
            <button type="submit">Send</button> # Submit button that sends message to B.E. (triggers
            # chat rout in app.py where B.E. processes message/generate response)
        </form>
        <form action="{{ url_for('terminate') }}" method="post">
            <input type="hidden" name="chat_id" value="{{ chat_id }}">
            <button type="submit">End Chat</button># Terminate button (redirect to home)
        </form>
    </div>
</body>
</html>


style.css

" You may not like him, Minister, but you can't deny: Dumbledore's got style."

This is used to style all the HTML pages

* General Layout/Containers
  - Centers content
  - Adds padding/applies max width in case of wide screens
  - Makes sure looks uniform
* Text/Font Styling
  - Basic header, font (size/weight/color) stuff
* Form/Button Styling
  - Padding, border-radius, width for search box/chat messages
  - Different color for placeholder text
  - Button styling to look clickable (colors, padding, hover effects)
* Message Disply Styling
  - User/assistant messages styled differently to tell the difference
* Navigation/Links Style
  - Links styled to be consistent with the rest of them (underline)
* Screen Compatibility
  - Attempts to make app usable on mobile/desktop

In [ ]:
/* style.css */

body {
    font-family: Arial, sans-serif;
    text-align: center;
    margin: 0;
    padding: 0;
    background-color: #f9f9f9;
}

.container {
    max-width: 600px;
    margin: 50px auto;
    padding: 20px;
    border-radius: 8px;
    background-color: #ffffff;
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
}

h1, h2 {
    color: #2c3e50;
}

input[type="text"], button {
    padding: 10px;
    margin-top: 10px;
    width: 80%;
    border: 1px solid #ddd;
    border-radius: 4px;
}

button {
    width: auto;
    background-color: #2980b9;
    color: white;
    cursor: pointer;
    border: none;
    margin-top: 10px;
}

button:hover {
    background-color: #3498db;
}

.results, #messages {
    text-align: left;
    margin-top: 20px;
}

.back-link, .end-chat {
    color: #2980b9;
    text-decoration: none;
    margin-top: 20px;
    display: inline-block;
}

.back-link:hover, .end-chat:hover {
    color: #3498db;
}
